In [ ]:
#docker run -e HZ_NETWORK_PUBLICADDRESS=192.168.0.105:5701 -p 5701:5701 -d hazelcast/hazelcast:latest
#docker run -e HZ_NETWORK_PUBLICADDRESS=192.168.0.105:5702 -p 5702:5701 -d hazelcast/hazelcast:latest
#docker run -e HZ_NETWORK_PUBLICADDRESS=192.168.0.105:5703 -p 5703:5701 -d hazelcast/hazelcast:latest

In [1]:
import hazelcast
import logging
import threading

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
client = hazelcast.HazelcastClient()

INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is STARTING
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is STARTED
INFO:hazelcast.connection:Trying to connect to Address(host=127.0.0.1, port=5701)
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is CONNECTED
INFO:hazelcast.connection:Authenticated with server Address(host=192.168.0.105, port=5701):78d11c9b-544d-416f-b77f-19dc93c74064, server version: 5.3.6, local address: Address(host=127.0.0.1, port=52916)
INFO:hazelcast.cluster:

Members [3] {
	Member [192.168.0.105]:5703 - 09710b69-50e7-4275-a36c-6d712a6a7a11
	Member [192.168.0.105]:5702 - 10c025ca-7c7a-4ed1-bbcf-37fdf10ee598
	Member [192.168.0.105]:5701 - 78d11c9b-544d-416f-b77f-19dc93c74064
}

INFO:hazelcast.connection:Authenticated with server Address(host=192.168.0.105, port=5703):09710b69-50e7-4275-a36c-6d712a6a7a11, server version: 5.3.6, local address: Address(host=192.168.0.105, port=52919)
INFO:hazelcast.connection:Authenticated with server Address(host=192.168.0.105, port=

In [4]:
distributed_map = client.get_map('distributed-map')
distributed_map.set('counter', 0).result()

In [5]:
distributed_map.get('counter').result()

0

In [16]:
def thread_function():
    for _ in range(10000):
        counter = distributed_map.get('counter').result()
        distributed_map.put('counter', counter+1).result()

In [17]:
threads = []

for index in range(10):
    print(f'Create and start thread {index}')
    x = threading.Thread(target=thread_function)
    threads.append(x)
    x.start()

for index, thread in enumerate(threads):
    print(f'Before joining thread {index}')
    thread.join()
    print(f'Thread {index} done')

Create and start thread 0
Create and start thread 1
Create and start thread 2
Create and start thread 3
Create and start thread 4
Create and start thread 5
Create and start thread 6
Create and start thread 7
Create and start thread 8
Create and start thread 9
Before joining thread 0
Thread 0 done
Before joining thread 1
Thread 1 done
Before joining thread 2
Thread 2 done
Before joining thread 3
Thread 3 done
Before joining thread 4
Thread 4 done
Before joining thread 5
Thread 5 done
Before joining thread 6
Thread 6 done
Before joining thread 7
Thread 7 done
Before joining thread 8
Thread 8 done
Before joining thread 9
Thread 9 done


In [18]:
distributed_map.get('counter').result()

10442

In [13]:
distributed_map.put('counter', 0).result()

39

In [9]:
def thread_function_two():
    for _ in range(10):
        distributed_map.lock('counter')
        counter = distributed_map.get('counter').result()
        distributed_map.put('counter', counter+1).result()
        distributed_map.force_unlock('counter')

In [10]:
thread_function_two()

In [16]:
threads = []

for index in range(4):
    print(f'Create and start thread {index}')
    x = threading.Thread(target=thread_function_two)
    threads.append(x)
    x.start()

for index, thread in enumerate(threads):
    print(f'Before joining thread {index}')
    thread.join()
    print(f'Thread {index} done')

Create and start thread 0
Create and start thread 1
Create and start thread 2
Create and start thread 3
Before joining thread 0


In [26]:
distributed_map.get('counter').result()

4658

In [39]:
distributed_map.put('counter', 0).result()

10000

In [40]:
def thread_function_three():
    for _ in range(10000):
        check = True
        while check:
            counter = distributed_map.get('counter').result()
            if distributed_map.replace_if_same('counter', counter, counter+1).result():
                check = False

In [41]:
threads = []

for index in range(10):
    print(f'Create and start thread {index}')
    x = threading.Thread(target=thread_function_three)
    threads.append(x)
    x.start()

for index, thread in enumerate(threads):
    print(f'Before joining thread {index}')
    thread.join()
    print(f'Thread {index} done')

Create and start thread 0
Create and start thread 1
Create and start thread 2
Create and start thread 3
Create and start thread 4
Create and start thread 5
Create and start thread 6
Create and start thread 7
Create and start thread 8
Create and start thread 9
Before joining thread 0
Thread 0 done
Before joining thread 1
Thread 1 done
Before joining thread 2
Thread 2 done
Before joining thread 3
Thread 3 done
Before joining thread 4
Thread 4 done
Before joining thread 5
Thread 5 done
Before joining thread 6
Thread 6 done
Before joining thread 7
Thread 7 done
Before joining thread 8
Thread 8 done
Before joining thread 9
Thread 9 done


In [43]:
distributed_map.get('counter').result()

100000